In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install transformers==2.1.0
!pip install gensim==4.1.2
!pip3 install --user underthesea

# !pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#from pyvi import ViTokenizer, ViPosTagger
import regex as re
from underthesea import word_tokenize
import numpy as np
import pandas as pd
import gensim
import sklearn
import tensorflow as tf
from sklearn import svm
from sklearn.model_selection import train_test_split
#from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import *
from keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.metrics import sparse_categorical_accuracy
from tensorflow.keras.optimizers import Adam
# Chia tập train/test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [ ]:
# Thống kê số lượng data theo nhãn
count = {}
for line in open('/content/drive/MyDrive/KhanhMD/Data/news_categories.csv'):
    key = line.split()[0]
    count[key] = count.get(key, 0) + 1

for key in count:
    print(key, count[key])

__label__thể_thao 2611
__label__âm_nhạc 2595
__label__nhịp_sống 2613
__label__công_nghệ 2595
__label__thời_sự 2607
__label__thế_giới 2602
__label__thời_trang 2596
__label__du_lịch 2593
__label__sống_trẻ 2602
__label__giáo_dục 2603
__label__kinh_doanh 2597
__label__pháp_luật 2592
__label__giải_trí 2604
__label__phim_ảnh 2596
__label__xe_360 2602
__label__ẩm_thực 2482
__label__xuất_bản 2599
__label__sức_khỏe 2589


In [ ]:
# Thống kê các word xuất hiện ở tất cả các nhãn
total_label = 18
vocab = {}
label_vocab = {}
for line in open('/content/drive/MyDrive/KhanhMD/Data/news_categories.csv'):
    words = line.split()
    # lưu ý từ đầu tiên là nhãn
    label = words[0]
    if label not in label_vocab:
        label_vocab[label] = {}
    for word in words[1:]:
        label_vocab[label][word] = label_vocab[label].get(word, 0) + 1
        if word not in vocab:
            vocab[word] = set()
        vocab[word].add(label)

# count = {}
# for word in vocab:
#     if len(vocab[word]) == total_label:
#         count[word] = min([label_vocab[x][word] for x in label_vocab])
        
# sorted_count = sorted(count, key=count.get, reverse=True)
# for word in sorted_count[:100]:
#     print(word, count[word])
text = []
label = []

for line in open('/content/drive/MyDrive/KhanhMD/Data/news_categories.csv'):
    words = line.strip().split()
    label.append(words[0])
    text.append(' '.join(words[1:]))

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(label)
encode_label = label_encoder.transform(label)

In [ ]:
import pandas as pd
df = pd.DataFrame(text, columns=["text"])
df.to_csv('text.csv', index=False)

In [ ]:
rd = pd.read_csv('text.csv')
rd[:10]

,text
0,sanchez không dự trận khai màn nh anh cùng ars...
1,những sao việt hóa_thân thành phó nháy trong m...
2,vợ phải quỳ gối mới được nói_chuyện cười zing ...
3,lumia 520 sẽ có giá 3 9 triệu đồng tại việt_na...
4,hà_nội t t chốt danh_sách dự afc cup thể_thao ...
5,vụ xe điên gây tai_nạn ở sài_gòn tài_xế bị tâm...
6,lính nga nhảy_dù xuống tảng băng trôi trên bắc...
7,ngắm vĩnh_thụy đầy chất chơi thời_trang zing v...
8,bố_mẹ dừng tàu lượn siêu_tốc vì lo cho con_gái...
9,cụ bà 84 tuổi đặc_trị bệnh chó dại thời_sự zin...


In [ ]:
import pandas as pd
df = pd.DataFrame(encode_label, columns=["label"])
df.to_csv('label.csv', index=False)

In [ ]:
lb = pd.read_csv('label.csv')
lb[:10]

,label
0,11
1,16
2,5
3,0
4,11
5,12
6,10
7,13
8,1
9,12


In [ ]:
a = pd.read_csv("label.csv")
b = pd.read_csv("text.csv")
df_merge_col = pd.merge(a, b,left_index=True, right_index=True, how='outer')
df_merge_col.to_csv("data.csv", index=False)

In [ ]:
c = pd.read_csv("data.csv")
c[:10]

,label,text
0,11,sanchez không dự trận khai màn nh anh cùng ars...
1,16,những sao việt hóa_thân thành phó nháy trong m...
2,5,vợ phải quỳ gối mới được nói_chuyện cười zing ...
3,0,lumia 520 sẽ có giá 3 9 triệu đồng tại việt_na...
4,11,hà_nội t t chốt danh_sách dự afc cup thể_thao ...
5,12,vụ xe điên gây tai_nạn ở sài_gòn tài_xế bị tâm...
6,10,lính nga nhảy_dù xuống tảng băng trôi trên bắc...
7,13,ngắm vĩnh_thụy đầy chất chơi thời_trang zing v...
8,1,bố_mẹ dừng tàu lượn siêu_tốc vì lo cho con_gái...
9,12,cụ bà 84 tuổi đặc_trị bệnh chó dại thời_sự zin...


In [ ]:
df = c.copy()
df[:10]

,label,text
0,11,sanchez không dự trận khai màn nh anh cùng ars...
1,16,những sao việt hóa_thân thành phó nháy trong m...
2,5,vợ phải quỳ gối mới được nói_chuyện cười zing ...
3,0,lumia 520 sẽ có giá 3 9 triệu đồng tại việt_na...
4,11,hà_nội t t chốt danh_sách dự afc cup thể_thao ...
5,12,vụ xe điên gây tai_nạn ở sài_gòn tài_xế bị tâm...
6,10,lính nga nhảy_dù xuống tảng băng trôi trên bắc...
7,13,ngắm vĩnh_thụy đầy chất chơi thời_trang zing v...
8,1,bố_mẹ dừng tàu lượn siêu_tốc vì lo cho con_gái...
9,12,cụ bà 84 tuổi đặc_trị bệnh chó dại thời_sự zin...


In [ ]:
df.shape

(46678, 2)

In [ ]:
#tok = Tokenizer()
# lst_corpus = []
# for string in df['text']:
#    lst_words = string.split()
#    lst_grams = [" ".join(lst_words[i:i+1]) for i in range(0, len(lst_words), 1)]
#    lst_corpus.append(lst_grams)

# print(len(lst_corpus))


In [ ]:
lst_corpus = [[word for word in document.split()] for document in text]

In [ ]:
len(lst_corpus)

46678

In [ ]:
w2v = gensim.models.fasttext.FastText(lst_corpus,
                                   min_n=2,
                                   bucket= 500000,
                                   vector_size=100,
                                   window=5,
                                   min_count=1)
words = set(w2v.wv.index_to_key)

In [ ]:
w2v.train(lst_corpus,epochs=10,total_examples=len(lst_corpus))

(227649413, 256614670)

In [ ]:
vocab=w2v.wv.key_to_index
print("The total number of words are : ",len(vocab))

The total number of words are :  233702


In [ ]:
vocab=list(vocab.keys())

In [ ]:
word_vec_dict={}
for word in vocab:
  word_vec_dict[word]=w2v.wv.get_vector(word)
print(len(word_vec_dict))

233702


In [ ]:
# maxi=-1
# for i,rev in enumerate(df['text']):
#     tokens=rev.split()
#     if(len(tokens)>maxi):
#         maxi=len(tokens)
# print(maxi)

In [ ]:
tok = Tokenizer()
tok.fit_on_texts(df['text'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(df['text'])

In [ ]:
max_rev_len=700  # max lenght of a review
vocab_size = len(tok.word_index) + 1  # total no of words
embed_dim=100

In [ ]:
from keras.preprocessing.sequence import pad_sequences
pad_rev= pad_sequences(encd_rev, maxlen=max_rev_len, padding='post')
pad_rev.shape 

(46678, 700)

In [ ]:
pad_rev

array([[4729,   10,  226, ...,    0,    0,    0],
       [   1, 3335,   95, ...,   53,  439,   58],
       [ 490,   54, 2801, ...,    0,    0,    0],
       ...,
       [ 354,  328,  376, ...,  120,  260,   55],
       [2610,  298,  692, ...,    0,    0,    0],
       [  39, 1143, 7507, ...,    0,    0,    0]], dtype=int32)

In [ ]:
embed_matrix=np.zeros(shape=(vocab_size,embed_dim))
for word,i in tok.word_index.items():
  embed_vector=word_vec_dict.get(word)
  if embed_vector is not None:  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

In [ ]:
embed_matrix.shape

(93363, 100)

In [ ]:
import keras
import keras.utils
from keras import utils as np_utils
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from keras.utils import np_utils
sklearn.model_selection.train_test_split

<function sklearn.model_selection._split.train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)>

In [ ]:
# !pip install np_utils
from keras.utils.np_utils import to_categorical

In [ ]:
# Y=keras.utils.to_categorical(df['label'])
# Y.shape
from keras import utils as np_utils
Y=keras.utils.np_utils.to_categorical(df['label'])  # one hot target as required by NN.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(pad_rev, Y, test_size=0.1, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42) # 0.25 x 0.8 = 0.2


In [ ]:
from keras.initializers import Constant
from keras.layers import ReLU
from keras.layers import Dropout
model=Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embed_dim,input_length=max_rev_len,embeddings_initializer=Constant(embed_matrix)))
model.add(Bidirectional(LSTM(128)))
model.add(Flatten())
model.add(Dense(64,activation='relu',kernel_regularizer='l2'))
model.add(Dropout(0.2))

#model.add(Dense(1024,activation='relu'))
# model.add(Dense(64,activation='relu',kernel_regularizer='l2'))
# model.add(Dropout(0.20))
# model.add(Dense(32,activation='relu',kernel_regularizer='l2'))
#model.add(Dense(32,activation='relu',kernel_regularizer='l2'))
# model.add(Dense(1,activation='relu'))
# model.add(Dense(16,activation='relu'))
# model.add(Dropout(0.20))
model.add(Dense(18,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 700, 100)          9336300   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              234496    
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 64)                16448     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 18)                1170      
                                                        

In [ ]:
from tensorflow import keras
from keras import optimizers

In [ ]:
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
epochs=30
batch_size=64

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [ ]:
model.fit(X_train,y_train,epochs=epochs,batch_size=batch_size,validation_data=(X_val,y_val),callbacks=[es, mc])

Epoch 1/30
526/526 [==============================] - ETA: 0s - loss: 3.4330 - accuracy: 0.2194
Epoch 1: val_loss improved from inf to 2.82831, saving model to best_model.h5
526/526 [==============================] - 63s 105ms/step - loss: 3.4330 - accuracy: 0.2194 - val_loss: 2.8283 - val_accuracy: 0.4010
Epoch 2/30
526/526 [==============================] - ETA: 0s - loss: 2.2995 - accuracy: 0.4979
Epoch 2: val_loss improved from 2.82831 to 1.73811, saving model to best_model.h5
526/526 [==============================] - 54s 103ms/step - loss: 2.2995 - accuracy: 0.4979 - val_loss: 1.7381 - val_accuracy: 0.6638
Epoch 3/30
526/526 [==============================] - ETA: 0s - loss: 1.6082 - accuracy: 0.6709
Epoch 3: val_loss improved from 1.73811 to 1.33249, saving model to best_model.h5
526/526 [==============================] - 56s 106ms/step - loss: 1.6082 - accuracy: 0.6709 - val_loss: 1.3325 - val_accuracy: 0.7489
Epoch 4/30
526/526 [==============================] - ETA: 0s - loss

In [ ]:
modelx = keras.models.load_model('best_model.h5')

In [ ]:
y_pred_one_hot_encoded = (modelx.predict(X_test)> 0.5).astype("float")
#y_pred_test = np.array(tf.argmax(y_pred_one_hot_encoded, axis=1))


In [ ]:
print(classification_report(y_test, y_pred_one_hot_encoded))

              precision    recall  f1-score   support

           0       0.98      0.91      0.94       268
           1       0.94      0.93      0.94       267
           2       0.92      0.80      0.85       270
           3       0.85      0.79      0.82       228
           4       0.91      0.81      0.86       270
           5       0.90      0.85      0.87       252
           6       0.88      0.91      0.89       250
           7       0.95      0.95      0.95       270
           8       0.79      0.71      0.75       249
           9       0.91      0.91      0.91       252
          10       0.94      0.90      0.92       258
          11       0.98      0.94      0.96       264
          12       0.92      0.84      0.88       241
          13       0.96      0.92      0.94       277
          14       0.98      0.97      0.97       262
          15       0.93      0.94      0.94       247
          16       0.95      0.87      0.91       290
          17       0.95    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
modelx.evaluate(X_test, y_test, batch_size=64)

73/73 [==============================] - 3s 33ms/step - loss: 0.4050 - accuracy: 0.9002


[0.40503913164138794, 0.9001713991165161]